    %load_ext pidgin.shell
    
The `pidgin` shell augments the `IPython` shell in the notebook to permit Markdown forward [computational essays](http://blog.stephenwolfram.com/2017/11/what-is-a-computational-essay/).   It provides:

* Markdown input in code cells.
* Markdown block code statements and expressions are evaluated as a normal code cell.
* Inline code expressions are evaluated asynchronously.
* The Markdown code accepts `jinja2` templates too


In [1]:
import IPython
try:
    import markdown
except:
    with __import__('importnb').Notebook(): from pidgin import markdown

import types, sys, IPython, asyncio, trio, ipykernel, traitlets, nbconvert, jinja2, operator, toolz.curried as toolz, tokenize, io, textwrap

ip = IPython.get_ipython()


In [2]:
def expression_tokens(str): return list(tokenize.generate_tokens(io.StringIO(str).readline))

In [3]:
def split_expression(str, *expressions):
    """Split an expression on the semi colons."""
    start = 0
    for id in toolz.pipe(
        str, expression_tokens, 
        toolz.filter(toolz.compose(
            ';'.__eq__, operator.attrgetter('string')
        )), 
        toolz.map(toolz.compose(
            toolz.second,
            operator.attrgetter('start')
        )),
        list
    ) + [len(str)]:
        expressions += str[start:id],
        start = id + 1
    return expressions

In [4]:
def requote(str, token='"""'):
    if token in str: token = "'''"
    return token+str+token

In [5]:
    environment = jinja2.Environment(enable_async=True)
    environment.globals.update(vars(__import__('builtins')))

In [24]:
    class PidginShell(IPython.core.interactiveshell.InteractiveShell):
        markdown = traitlets.Bool(True)
        template = traitlets.Bool(True)
        environment = traitlets.Instance(
            jinja2.Environment, 
            default_value=environment)
        expressions = traitlets.Bool(True)
        tangle_expressions = traitlets.Bool(True)

        async def single_user_expression(self, code):
            expressions = split_expression(code)
            results = IPython.core.interactiveshell.InteractiveShell.user_expressions(self, dict(
                zip(expressions, map(self.input_transformer_manager.transform_cell, expressions))))

            IPython.display.display(IPython.display.Markdown('''`>>> {}`'''.format(code.strip('`'))))

            error_msg = []
            for expression, result in results.items():
                expressions_ns = '`{}`'.format(expression.strip())
                if result['status'] == 'error':
                    error_msg.extend(result['traceback'])
                elif expressions_ns in self.user_ns:
                    display = self.user_ns[expressions_ns]
                    IPython.display.publish_display_data(result['data'], update=True, transient={'display_id': display.display_id})

            if error_msg:
                IPython.display.publish_display_data({'text/plain': ''.join(error_msg)})

            if result['status'] == 'ok':
                display = self.user_ns[
                    "`{}`".format(code.strip())
                ] = self.user_ns[expressions_ns] = IPython.display.DisplayHandle()
                IPython.display.publish_display_data(result['data'], transient={
                    'display_id': display.display_id})
            if error_msg: result['status'] = 'error'
            return result

        def run_cell(self, code, store_history=False, silent=False, shell_futures=True, **user_expressions):
            """Run cell separates the sync and async parts."""
            self._last_traceback = None
            
            if self.markdown and not code.lstrip().startswith('%'): 
                source = markdown.renderer(code, user_expressions=user_expressions)
            else: source = code

            silent = silent or not source
            display = None

            if self.markdown:
                if code.strip() and code.splitlines()[0].strip():
                    if source and textwrap.dedent(code).strip() != source.strip():
                        display = IPython.display.display(IPython.display.Markdown(code), display_id=True)

            if not source:
                if self.template:
                    code = self.environment.from_string(code).render(**self.user_ns)
                result = IPython.core.interactiveshell.InteractiveShell.run_cell(
                        self, requote(code), store_history=False, silent=silent, shell_futures=shell_futures)        
            else: 
                result = IPython.core.interactiveshell.InteractiveShell.run_cell(
                    self, source, store_history=store_history, silent=silent, shell_futures=shell_futures)
            
            trio.run(self.async_run_cell, code, display, store_history, silent, shell_futures, user_expressions)
            
            return result
        
        async def async_run_cell(self, code, display, store_history=False, silent=False, shell_futures=True, user_expressions=None):
            async with trio.open_nursery() as nursery:
                if user_expressions:
                    IPython.display.display(IPython.display.Markdown('---'))
                    for key, expression in user_expressions.items():
                        nursery.start_soon(self.single_user_expression, expression)

                if display and self.template: 
                    nursery.start_soon(self.update_display_template, display, code)

        
        async def update_display_template(self, display, code):
            display.update(IPython.display.Markdown(
                await self.environment.from_string(code).render_async(**self.user_ns)
            ))

In [25]:
    original_methods = {}
    def load_ipython_extension(ip):
        global original_methods
        for method in (
            PidginShell.run_cell, PidginShell.async_run_cell, 
            PidginShell.single_user_expression, 
            PidginShell.update_display_template
        ): 
            object = getattr(ip, method.__name__, None)
            if object:
                original_methods[method.__name__] = original_methods.get(method.__name__, object)
            setattr(ip, method.__name__, types.MethodType(method, ip))

        for trait in (
            'markdown', 'template', 'expressions', 'tangle_expressions', 'environment'
        ): 
            ip.add_traits(**{trait: getattr(PidginShell,  trait)})
            setattr(ip, trait, getattr(PidginShell,  trait).default_value)

    def unload_ipython_extension(ip): ip.__dict__.update(original_methods)


In [26]:
    if __name__ == '__main__':
        #!ipython -m pytest -- shell.ipynb
        load_ipython_extension(get_ipython())

Now this is a test {{time.sleep(2) or ip.user_ns.update(a=100)}} `time.sleep(2) or ip.user_ns.update(a=1000)` {{a}}

`'revel'`
    
    import time
    a = time.sleep(1) or 900
    print('xxx')

In [28]:
    def test_nbconvert():
        get_ipython = IPython.get_ipython
        md = !jupyter nbconvert --to markdown --execute --stdout shell.ipynb        

In [29]:
    def test_extension():
        ip = IPython.get_ipython()
        try:
            ip.run_cell("""This is not code\n\n\n\ta=range(10)""")
            assert False
        except: assert True
        load_ipython_extension(ip)
        ip.run_cell("""This is not code\n\n\n\ta=range(10)""")
        unload_ipython_extension(ip)

In [31]:
    import ast

In [36]:
    list(tokenize.generate_tokens(io.StringIO('"do it" = 11').readline))

[TokenInfo(type=3 (STRING), string='"do it"', start=(1, 0), end=(1, 7), line='"do it" = 11'),
 TokenInfo(type=53 (OP), string='=', start=(1, 8), end=(1, 9), line='"do it" = 11'),
 TokenInfo(type=2 (NUMBER), string='11', start=(1, 10), end=(1, 12), line='"do it" = 11'),
 TokenInfo(type=0 (ENDMARKER), string='', start=(2, 0), end=(2, 0), line='')]